### Imports
 Primeiramente são importadas as bibliotecass
nescessárias, sendo elas:
 - pandas: para leitura dos dados da base de dados
 - sklearn model_selection: para seleção dos modelos
 - optuna: para optimazação de hiperparametros
 
    As demais são os métodos de regressão escolhidos:
 - Floresta Aleatória (Regressão)

In [155]:
import pandas as pd
import numpy as np
np.set_printoptions(legacy='1.25')

from sklearn.ensemble import RandomForestRegressor
from sklearn import model_selection
from sklearn.model_selection import KFold

import optuna
from sklearn.model_selection import cross_val_score

### Base de Dados
Então realiza-se a leitura da base de dados contida no arquivo 'Analise Geral Normalizada.xlsx' e seus dados são divididos entre:
- X : dados de entrada da função.
- Y : valor a ser predito

In [156]:
df = pd.read_excel("Analise Geral Normalizada.xlsx")
array = df.values
X = array[:, :22]
y = array[:,22]
df.head()

,Vel. Vento (m/s) PoA,Dir. Vento (m/s) PoA,Chuva (mm) PoA,Nível adotado (cm) Guaiba,Chuva Taquari (cm),Nivel Taquari (m),Vazão Taquari (Mm3/s),Chuva Jacui (mm),Nivel Jacui (m),Vazão Jacui (Mm3/s),...,Chuva (cm) soma 8h Rio Pardo,Vel. Vento (m/s) média 8h Soledade,Dir. Vento (m/s) média 8h Soledade,Raj. Vento (m/s) média 8h Soledade,Chuva (cm) soma 8h Soledade,Vel. Vento (m/s) média 8h Bento,Dir. Vento (m/s) média 8h Bento,Raj. Vento (m/s) média 8h Bento,Chuva (cm) soma 8h Bento,Nível após uma semana (m)
0,0.2258,0.8889,0.0,0.2392,0.0,0.1499,0.0819,0.0,0.3241,0.2084,...,0.0,0.0402,0.7883,0.1456,0.0,0.0668,0.6186,0.0349,0.0,0.1478
1,0.1075,0.7500,0.0,0.2523,0.0,0.1534,0.0845,0.0,0.3449,0.2249,...,0.0,0.0013,0.4181,0.0366,0.0,0.4480,0.1275,0.3867,0.0,0.1425
2,0.1075,0.7500,0.0,0.2704,0.0,0.1618,0.0909,0.0,0.3554,0.2333,...,0.0,0.0000,0.2016,0.0000,0.0,0.3193,0.0898,0.1940,0.0,0.1747
3,0.3333,0.8889,0.0,0.2913,0.0,0.1704,0.0977,0.0,0.3658,0.2417,...,0.0,0.0665,0.8504,0.2277,0.0,0.1782,0.7920,0.1819,0.0,0.1934
4,0.3333,0.7500,0.0,0.3198,0.0,0.1821,0.1071,0.0,0.3898,0.2616,...,0.0,0.0414,0.4458,0.0952,0.0,0.0718,0.4771,0.0265,0.0,0.1553


TREINAMENTO

In [157]:
randomForest = RandomForestRegressor(random_state=2024)

In [158]:
kfold = KFold(n_splits=5, shuffle=False)

In [159]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=2024)

### Analise inicial
São feitos alguns testes para analisar o funcionamento inicial da arvore com os dados disponiveis

In [160]:
randomForest.fit(X_train, y_train)

RandomForestRegressor(random_state=34)

In [161]:
y_pred = randomForest.predict(X_test)
randomForest.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 34,
 'verbose': 0,
 'warm_start': False}

In [162]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [163]:
mean_absolute_error(y_test, y_pred)

0.03738422857142849

In [164]:
mean_squared_error(y_test, y_pred)

0.0022248295604571366

In [165]:
r2_score(y_test, y_pred)

0.9732253199629749

Pode se perceber que a floresta já possui saídas bastante satisfátórias com um erro quádratico médio bastante baixo e um score r2 bastante alto, portanto encontrar definições de hiperparametros melhores pode ser dificil

### OPTUNA e HIPERPARAMETROS
Agora são feitos testes com o frame optuna para descobrir quais os melhores hiperparametros para a função, sendo que os hiperparametros analisados foram:

- n_estimators: Número de árvores na floresta.
- max_depth: Tamanho máximo das arvores individuais.
- min_smaples_split: Número mínimo de amostras necessárias para dividir um nó.
- min_samples_leaf: Número mínimo de amostras necessárias em um nó folha.


In [166]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 1, 15)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 15)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 15)

    model = RandomForestRegressor(n_estimators=n_estimators,
    max_depth=max_depth,
    min_samples_split=min_samples_split,
    min_samples_leaf=min_samples_leaf)

    score = cross_val_score(model, X, y, n_jobs=-1, cv=kfold, scoring='neg_mean_squared_error').mean()

    return score

In [167]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=2024))

[I 2024-08-18 10:21:16,188] A new study created in memory with name: no-name-e3f8c5b6-9930-4a83-b048-4a0b730e6f3c


In [168]:
study.optimize(objective, n_trials=400)

[I 2024-08-18 10:21:19,782] Trial 0 finished with value: -0.029335939098191155 and parameters: {'n_estimators': 138, 'max_depth': 11, 'min_samples_split': 4, 'min_samples_leaf': 1}. Best is trial 0 with value: -0.029335939098191155.
[I 2024-08-18 10:21:20,120] Trial 1 finished with value: -0.036829810610052124 and parameters: {'n_estimators': 80, 'max_depth': 2, 'min_samples_split': 12, 'min_samples_leaf': 11}. Best is trial 0 with value: -0.029335939098191155.
[I 2024-08-18 10:21:20,714] Trial 2 finished with value: -0.03641814857107408 and parameters: {'n_estimators': 121, 'max_depth': 7, 'min_samples_split': 2, 'min_samples_leaf': 12}. Best is trial 0 with value: -0.029335939098191155.
[I 2024-08-18 10:21:21,464] Trial 3 finished with value: -0.03286014570861765 and parameters: {'n_estimators': 140, 'max_depth': 15, 'min_samples_split': 11, 'min_samples_leaf': 10}. Best is trial 0 with value: -0.029335939098191155.
[I 2024-08-18 10:21:22,139] Trial 4 finished with value: -0.03574598

In [169]:

best_params = study.best_params
best_score = study.best_value
print(f"Best Hyperparameters: {best_params}")
print(f"Best Accuracy: {best_score:.4f}")

Best Hyperparameters: {'n_estimators': 121, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 4}
Best Accuracy: -0.0280


In [170]:
optuna.visualization.plot_optimization_history(study)

In [171]:
optuna.visualization.plot_slice(study, params=['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split'])

Aqui pode se perceber que min_samples_leaf e max_depth possuem uma influencia maior sobre a saída sendo valores mais baixos melhores para min_samples_leaf e valores a partir de 5 para max_depth, pois podemos ver uma certa ordem nas saidas, enquanto max_depth e n_estimators praticamente não possuem ordem

In [172]:
optuna.visualization.plot_param_importances(study)

Expresso aqui também a importancia de min_samples_leaf e min_samples_split

### AVALIAÇÂO FINAL

In [173]:
best_n_estimators = best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']


florestaFinal = RandomForestRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth, min_samples_split=best_min_samples_split, min_samples_leaf=best_min_samples_leaf)


In [174]:
florestaFinal.fit(X_train, y_train)
y_pred = florestaFinal.predict(X_test)

In [175]:

mean_absolute_error(y_test, y_pred)


0.04101953056886509

In [176]:
mean_squared_error(y_test, y_pred)

0.0028952276448947966

In [177]:

r2_score(y_test, y_pred)

0.9651574236498007